In [ ]:
import numpy as np
import pandas as pd

In [ ]:
DATASET_PATH = "datasets/"

In [ ]:
import os

In [ ]:
movies = pd.read_csv(os.path.join(DATASET_PATH, "tmdb_5000_movies.csv"))
credit = pd.read_csv(os.path.join(DATASET_PATH, "tmdb_5000_credits.csv"))

In [ ]:
movies = movies.merge(credit, on="title")

In [ ]:
movies.columns

In [ ]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'vote_average', 'vote_count', 'popularity', 'release_date', 'cast']]


movies.columns = map(str.lower, movies.columns)

In [ ]:
movies['genres'] = movies['genres'].apply(eval)
movies['genres'] = movies['genres'].apply(lambda x: [d['name'] for d in x])

In [ ]:
movies['keywords'] = movies['keywords'].apply(eval)
movies['keywords'] = movies['keywords'].apply(lambda x: [d['name'] for d in x])

In [ ]:
movies.head()

In [ ]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

In [ ]:
movies['cast'] = movies['cast'].apply(eval)
movies['cast'] = movies['cast'].apply(lambda x: [d['name'] for d in x])

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
movies.sample(5)

In [ ]:
# converting genres, keywords and cast to string 
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x))
movies['cast'] = movies['cast'].apply(lambda x: ' '.join(x))
movies['overview'] = movies['overview'].apply(lambda x: ' '.join(x))

In [ ]:
# making final column for training
movies['combined'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['title'] + movies['release_date'].apply(lambda x: str(x.year) + " " + str(x.month) + " " + str(x.day))
movies['title_lower'] = movies['title'].apply(lambda x: x.lower())

In [ ]:
movies

Machine Learning part

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
tf = TfidfVectorizer(max_features=5000, stop_words='english')

In [ ]:
tf_vector = tf.fit_transform(movies['combined']).toarray()

In [ ]:
sig = sigmoid_kernel(tf_vector, tf_vector)

In [ ]:
def recommend(movie):
    index = movies[movies['title_lower'] == movie.lower()].index[0]

    if index < 0:
        return []

    sig_scores = list(enumerate(sig[index]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_index = [i[0] for i in sig_scores]
    return movies[
        ['title', 'vote_count', 'vote_average', 'release_date', 'genres']
    ].iloc[movie_index].to_dict('records')

In [ ]:
recommend('interstellar')

In [ ]:
# using bayesian average
C = movies['vote_average'].mean()
m = movies['vote_count'].quantile(0.9)
v = movies['vote_count']
R = movies['vote_average']

movies['weighted_average'] = (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
def get_top_x_movies(x):
    # return list of top x movies with id, title, vote_count, vote_average, weighted_average, popularity
    return movies[['id', 'title', 'vote_count', 'vote_average', 'weighted_average', 'popularity']].sort_values('weighted_average', ascending=False).head(x).to_dict('records')

In [ ]:
def get_top_x_movies_by_genre(genre, x=10):
    return movies[movies['genres'].str.contains(genre)].sort_values('weighted_average', ascending=False).head(x).to_dict('records')


In [ ]:
get_top_x_movies_by_genre('action', 10)

In [ ]:
get_top_x_movies(10)

In [ ]:
# extracting movies df to be loaded as a df in another app and used for recommendation but not in csv format
movies[['id', 'title', 'vote_count', 'vote_average', 'weighted_average', 'popularity']].sort_values('weighted_average', ascending=False).head(10).to_dict('records')

# saving movies df to csv
movies[['id', 'title', 'vote_count', 'vote_average', 'weighted_average', 'popularity']].sort_values('weighted_average', ascending=False).head(10).to_csv('movies.csv', index=False)